In [40]:
from dotenv import load_dotenv

load_dotenv()

True

---

## LLM Chain

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
)

In [4]:
out = llm.invoke("how can langsmith help with testing?")

In [5]:
out

AIMessage(content='Langsmith can help with testing by providing a platform for automated testing of code written in various programming languages. It can help streamline the testing process by providing tools for writing test cases, running tests, and generating reports on test results. Additionally, Langsmith can help identify and fix bugs in the code by providing debugging tools and feedback on test failures. Overall, Langsmith can improve the efficiency and effectiveness of testing efforts by providing a centralized platform for managing and executing tests.')

In [7]:
out.content

'Langsmith can help with testing by providing a platform for automated testing of code written in various programming languages. It can help streamline the testing process by providing tools for writing test cases, running tests, and generating reports on test results. Additionally, Langsmith can help identify and fix bugs in the code by providing debugging tools and feedback on test failures. Overall, Langsmith can improve the efficiency and effectiveness of testing efforts by providing a centralized platform for managing and executing tests.'

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [9]:
chain = prompt | llm

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. **Automated Testing:** Langsmith can generate test cases based on the grammar rules provided to it. These test cases can be used to automate testing processes, ensuring comprehensive test coverage and reducing the manual effort required for testing.\n\n2. **Boundary Testing:** Langsmith can generate input values at the boundaries of different data ranges, helping testers identify potential boundary-related issues in the software under test.\n\n3. **Negative Testing:** Langsmith can generate test cases that include invalid or unexpected input values, allowing testers to verify how the software handles such scenarios.\n\n4. **Regression Testing:** Langsmith can help in generating regression test cases quickly and efficiently, ensuring that new code changes do not break existing functionality.\n\n5. **Load Testing:** By generating a large number of test cases, Langsmith can assist in load testing to evaluate the per

In [11]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [12]:
chain = prompt | llm | output_parser

In [13]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith is a powerful tool that can significantly aid in various aspects of testing. Here are some ways Langsmith can help with testing:\n\n1. **Automated Testing:** Langsmith can be used to automatically generate test cases for software applications. By providing input-output examples or specifications, Langsmith can generate a wide range of test cases to ensure comprehensive test coverage.\n\n2. **Regression Testing:** Langsmith can help in generating regression test cases quickly and efficiently. When new code is added or existing code is modified, Langsmith can be used to automatically generate test cases to ensure that the changes do not introduce any regression issues.\n\n3. **Boundary Testing:** Langsmith can help in generating boundary test cases to validate the behavior of software at the boundaries of input ranges. This can help in identifying potential issues related to boundary conditions that may not be covered by regular test cases.\n\n4. **Exploratory Testing:** Langs

---

## Retrieval Chain

In [17]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

In [18]:
docs = loader.load()

In [23]:
docs

[Document(page_content="\n\n\n\n\nLangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Set

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
)

In [15]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter()

In [24]:
documents = text_splitter.split_documents(docs)
documents

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [25]:
vector = FAISS.from_documents(documents, embeddings)

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [27]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [28]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [30]:
# By the way...

# If we wanted to, we could run this ourselves by passing in documents directly:

from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

In [31]:
# However, we want the documents to first come from the retriever we just set up.
# That way, for a given question we can use the retriever to dynamically select the most relevant documents and 
# pass those in.

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [32]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
response

{'input': 'how can langsmith help with testing?',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith support

In [33]:
response["answer"]

'LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It provides tracing and evaluation capabilities to assist with testing LLM applications.'

---

## Conversation Retrieval Chain

In [34]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [35]:
# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

In [36]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [37]:
# We can test this out by passing in an instance where the user is asking a follow up question.

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [39]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following s

---

## Agent

**NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.**